In [24]:
import re
import nltk
import pandas as pd
from textblob import Word
import numpy as np
import csv
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras import layers
from keras.layers import  Dropout, Dense
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from google.colab import files
uploaded = files.upload()

Saving dataset01.txt to dataset01.txt


In [4]:
data = pd.read_csv('dataset01.txt', sep = '\n')
print(data.head())

                                            all_data
0                                     >iTELL-Chr1_61
1  CACACCACACCCACACACCCACACACCACACCACACACCACACCAC...
2                                                  0
3                                   >iYAL067W-A_5085
4  AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCT...


In [5]:
all=data["all_data"].tolist()
cls=[]
sequence=[]
label=[]
all

['>iTELL-Chr1_61',
 'CACACCACACCCACACACCCACACACCACACCACACACCACACCACACCCACACACACACATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTGGCCAACCTGTCTCTCAACTTACCCTCCATTACCCTGCCTCCACTCGTTACCCTGTCCCATTCAACCATACCACTCCGAACCACCATCCATCCCTCTACTTACTACCACTCACCCACCGTTACCCTCCAATTACCCATATCCAACCCACTGCCACTTACCCTACCATTACCCTACCATCCACCATGACCTACTCACCATACTGTTCTTCTAC',
 '0',
 '>iYAL067W-A_5085',
 'AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCTTGTGCTTTCTTAGTATCTATCACAAAGGAGACAAAATCGTTGATAAAAAGTGCATCAACATTCCCAGCCAGAAAATGCACATCATAAAGACATGTTATTCAAGAGCCACGACCGTCTTCAATTTATCTTTTATAAAAAACCCTTGTTCTACTGACAGGATGGAATAGATATTAAATATACATTTTGCATTTTTTTTTTTTTCTGTATTGAAGATTTGTATATGAAAGATGTTTATACATCAAATGCTTTGAATAAAGCCATCTTAATTTCAATTTCATGCCCTCCTTCACCGTTTTCTGTTGGTCTAGAGGTAGCTTGTTGTGGTCACTAATGAGAACTTAAATAGTTTTCAACTGCTGGTGGTAAATCAATAATTTATGTTCTTAACCTAACATTTGATGACCTTTGATGCGTTGGTTATGTTGAAGACAAATTGCCTCTAATCAGTTCCATT',
 '0',
 '>iYAL067W-A_6786',
 'AATTATATTTCCATCAGCTCAATACCGCAGTACTTTGAAACCTGATTTATATATTGCAGAACTTAATTAAAAGTACATTGTAGTTCAAAAAATAAATATCAAACT

In [6]:
length = len(all)
for i in range(length):
   if (i%3)==0:
             cls.append(all[i])
   else:
      if(all[i]== "0" or all[i]=="1"):
        label.append(all[i])
      else:
         sequence.append(all[i])

In [7]:
print(len(cls))
print(len(label))
print(len(sequence))

14965
14965
14965


In [8]:
df = pd.DataFrame(list(zip(cls, label, sequence)),
               columns =['class' , 'label', 'sequence'])
df.head(10)

,class,label,sequence
0,>iTELL-Chr1_61,0,CACACCACACCCACACACCCACACACCACACCACACACCACACCAC...
1,>iYAL067W-A_5085,0,AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCT...
2,>iYAL067W-A_6786,0,AATTATATTTCCATCAGCTCAATACCGCAGTACTTTGAAACCTGAT...
3,>iYAL067W-A_7060,0,AACAATAGTGGGTATGAGTAAAGATATATAGATCGATATTTTGAAT...
4,>YAL067C_SEO1_8061,1,CAAAGATTTCAACCATAGTAGGTATGCCCCAGATGAAACATTACTG...
5,>YAL067C_SEO1_8340,1,CACAATAGCGTCAATAATAAAGTTCCATCTCCATCCCTCTAAACCA...
6,>iYAL067C_9509,1,GTTTTTTTCCAGTGACACAATCTTTACCATTACACAGTTTTTACTA...
7,>iYAL067C_10442,0,CAGAAACATAAAATTATATCACTTTATTTCATATGGTTTCATGCTT...
8,>YAL065C_YAL065C_11570,1,AGATACCTCGAAACAAACTCTATGTAAACACTTATTTTATTGTGGT...
9,>iYAL064W-B_13286,1,ATGACTGGAGAAGAAGTGACTAGAGAAGAAGTGACTGAGGAAGAAA...


In [12]:
X = df['sequence']
Y = df['label']

In [13]:
def getKmers(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

In [14]:
df['words'] = df.apply(lambda x: getKmers(x['sequence']), axis=1)
df = df.drop('sequence', axis=1)

In [19]:
df.head()

,class,label,words
0,>iTELL-Chr1_61,0,"[cacacc, acacca, caccac, accaca, ccacac, cacac..."
1,>iYAL067W-A_5085,0,"[aatttt, attttt, ttttta, ttttat, tttata, ttata..."
2,>iYAL067W-A_6786,0,"[aattat, attata, ttatat, tatatt, atattt, tattt..."
3,>iYAL067W-A_7060,0,"[aacaat, acaata, caatag, aatagt, atagtg, tagtg..."
4,>YAL067C_SEO1_8061,1,"[caaaga, aaagat, aagatt, agattt, gatttc, atttc..."


In [20]:
dna_texts = list(df['words'])
for item in range(len(dna_texts)):
    dna_texts[item] = ' '.join(dna_texts[item])
y_data = df['label'].values

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(4,4))
X = cv.fit_transform(dna_texts)
print(X.shape)

(14965, 261260)


In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y_data, test_size = 0.20,  random_state=42) 

print(X_train.shape)
print(X_test.shape)                                         

(11972, 261260)
(2993, 261260)


In [30]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500, random_state=0)
model.fit(X_train, y_train) 
model_predictions = model.predict(X_test)
print("\n---------Random Forest classifier---------")
print("\nConfusion matrix: \n", confusion_matrix(y_test,model_predictions))
print("\nClassification Report: \n", classification_report(y_test,model_predictions))
print("\nClassification accuracy: " ,accuracy_score(y_test, model_predictions) * 100, "%.\n")


---------Random Forest classifier---------

Confusion matrix: 
 [[1396   70]
 [ 612  915]]

Classification Report: 
               precision    recall  f1-score   support

           0       0.70      0.95      0.80      1466
           1       0.93      0.60      0.73      1527

    accuracy                           0.77      2993
   macro avg       0.81      0.78      0.77      2993
weighted avg       0.81      0.77      0.77      2993


Classification accuracy:  77.21349816237888 %.



In [31]:
from sklearn.naive_bayes import MultinomialNB
model_MNB = MultinomialNB(alpha=0.1)
model_MNB.fit(X_train, y_train)
model_predictions = model_MNB.predict(X_test)
print("\n---------Multinomial Naive Bayes---------")
print("\nConfusion matrix: \n", confusion_matrix(y_test,model_predictions))
print("\nClassification Report: \n", classification_report(y_test,model_predictions))
print("\nClassification accuracy: " ,accuracy_score(y_test, model_predictions) * 100, "%.\n")


---------Multinomial Naive Bayes---------

Confusion matrix: 
 [[1238  228]
 [ 150 1377]]

Classification Report: 
               precision    recall  f1-score   support

           0       0.89      0.84      0.87      1466
           1       0.86      0.90      0.88      1527

    accuracy                           0.87      2993
   macro avg       0.87      0.87      0.87      2993
weighted avg       0.87      0.87      0.87      2993


Classification accuracy:  87.37053123955897 %.



In [32]:
from sklearn import tree
model_DT = tree.DecisionTreeClassifier()
model_DT.fit(X_train, y_train)
model_predictions = model_DT.predict(X_test)
print("\n---------Decision tree classifier--------- ")
print("\nConfusion matrix: \n", confusion_matrix(y_test,model_predictions))
print("\nClassification Report: \n", classification_report(y_test,model_predictions))
print("\nClassification accuracy: " ,accuracy_score(y_test, model_predictions) * 100, "%.\n")


---------Decision tree classifier--------- 

Confusion matrix: 
 [[1108  358]
 [ 581  946]]

Classification Report: 
               precision    recall  f1-score   support

           0       0.66      0.76      0.70      1466
           1       0.73      0.62      0.67      1527

    accuracy                           0.69      2993
   macro avg       0.69      0.69      0.69      2993
weighted avg       0.69      0.69      0.68      2993


Classification accuracy:  68.62679585699966 %.



In [33]:
from sklearn.neighbors import KNeighborsClassifier
model_KNN = KNeighborsClassifier(n_neighbors=2)
model_KNN.fit(X_train, y_train) 
model_predictions = model_KNN.predict(X_test)
print("\n---------KNN classifier---------")
print("\nConfusion matrix: \n", confusion_matrix(y_test,model_predictions))
print("\nClassification Report: \n", classification_report(y_test,model_predictions))
print("\nClassification accuracy: " ,accuracy_score(y_test, model_predictions) * 100, "%.\n")


---------KNN classifier---------

Confusion matrix: 
 [[1460    6]
 [1264  263]]

Classification Report: 
               precision    recall  f1-score   support

           0       0.54      1.00      0.70      1466
           1       0.98      0.17      0.29      1527

    accuracy                           0.58      2993
   macro avg       0.76      0.58      0.49      2993
weighted avg       0.76      0.58      0.49      2993


Classification accuracy:  57.567657868359504 %.



In [34]:
from sklearn import svm
model_SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
model_SVM.fit(X_train, y_train) 
model_predictions = model_SVM.predict(X_test)
print("\n---------Support Vector Machine classifier---------")
print("\nConfusion matrix: \n", confusion_matrix(y_test,model_predictions))
print("\nClassification Report: \n", classification_report(y_test,model_predictions))
print("\nClassification accuracy: " ,accuracy_score(y_test, model_predictions) * 100, "%.\n")


---------Support Vector Machine classifier---------

Confusion matrix: 
 [[1271  195]
 [ 195 1332]]

Classification Report: 
               precision    recall  f1-score   support

           0       0.87      0.87      0.87      1466
           1       0.87      0.87      0.87      1527

    accuracy                           0.87      2993
   macro avg       0.87      0.87      0.87      2993
weighted avg       0.87      0.87      0.87      2993


Classification accuracy:  86.9695957233545 %.



In [35]:
from sklearn.linear_model import LogisticRegression
model_LR = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=2, penalty='l2',max_iter=1000)
model_LR.fit(X_train, y_train) 
model_predictions_LR = model_LR.predict(X_test)
print("\n---------Logistic regression classifier---------")
print("\nConfusion matrix: \n", confusion_matrix(y_test,model_predictions_LR))
print("\nClassification Report: \n", classification_report(y_test,model_predictions_LR))
print("\nClassification accuracy: " ,accuracy_score(y_test, model_predictions_LR) * 100, "%.\n")

[LibLinear]
---------Logistic regression classifier---------

Confusion matrix: 
 [[1273  193]
 [ 190 1337]]

Classification Report: 
               precision    recall  f1-score   support

           0       0.87      0.87      0.87      1466
           1       0.87      0.88      0.87      1527

    accuracy                           0.87      2993
   macro avg       0.87      0.87      0.87      2993
weighted avg       0.87      0.87      0.87      2993


Classification accuracy:  87.20347477447378 %.

